In [1]:
import pandas as pd
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
import smtplib

In [ ]:
#sending email command python
#editing for some parameters needed before running
conn=smtplib.SMTP('smtp.gmail.com', 587)
conn.ehlo()
conn.starttls()
conn.login('jyun1223334444@gmail.com', "ztszlgayszdmpbhe")
conn.sendmail('jyun1223334444@gmail.com', 'example@example.com', 'content goes here')

In [ ]:
#h5 file to csv
with pd.HDFStore("train_cite_inputs.h5", "r") as train:
    train_df=train.get("train_cite_inputs")
    train_df_short=train_df.head(1000)
#normal data to csv
train_df.to_csv('train_cite_inputs.csv')
#shortened data to csv
train_df_short.to_csv('short_train_cite_inputs.csv')

with pd.HDFStore("train_cite_targets.h5", "r") as train:
    train_df=train.get("train_cite_targets")
    train_df_short=train_df.head(1000)
#normal data to csv
train_df.to_csv('train_cite_targets.csv')
#shortened data to csv
train_df_short.to_csv('short_train_cite_targets.csv')

In [ ]:
#read in whole dataset
df_inputs = pd.read_csv('train_cite_inputs.csv')
df_targets = pd.read_csv('train_cite_targets.csv')
#read in short dataset
df_inputs_short = pd.read_csv('short_train_cite_inputs.csv')
df_targets_short = pd.read_csv('short_train_cite_targets.csv')

In [ ]:
#delete unnessesary columns
del df_inputs['cell_id']
del df_targets['cell_id']

del df_inputs_short['cell_id']
del df_targets_short['cell_id']

In [ ]:
#one random forest
X_train, X_test, y_train, y_test = train_test_split(df_inputs_short, df_targets_short['CD86'], test_size=0.2, random_state=0)
regressor=RandomForestRegressor(n_estimators = 100, max_depth=3)
regressor.fit(X_train, y_train)
#scoring for one random forest
y_hat=regressor.predict(X_test)
score=regressor.score(X_test, y_test)
print(score)

In [ ]:
#Run random forest for all columns
#Fix df_inputs to short_df_inputs etc. if you would want to run it on the short dataset
#Define list and dictionary
protein_list=list(df_targets.columns)
protein_model_dictionary = {}
#for loop to run model per column
for i in range(len(df_targets.columns)):
    protein_name=protein_list[i]
    #training data
    X_train, X_test, y_train, y_test = train_test_split(df_inputs, df_targets[protein_name], test_size=0.2, random_state=0)
    regressor=RandomForestRegressor(n_estimators=10, max_depth=5, random_state=0)
    regressor.fit(X_train, y_train, sample_weight=None)
    #getting score of model
    y_hat=regressor.predict(X_test)
    score=regressor.score(X_test, y_test)
    #storing in dictionary [keys:value] = [protein name:model]
    protein_model_dictionary[protein_name] = regressor
    print(i, ',', protein_name, ',', score)